In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import string
import random
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.contrib import seq2seq

from sklearn.utils import shuffle



/home/florian/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
epochs = 20000
lstm_size = 256
embed_size = 32
max_word_length = 12

In [3]:
with open('google-10000-english-usa-no-swears.txt', 'r') as word_file:
    words = word_file.read().split('\n')
    
# do a little bit of cleaning just in case
def only_letters(some_string):
    return ''.join([c for c in some_string if c in string.ascii_letters])

def pad_front(some_word):
    while len(some_word) < max_word_length:
        some_word = '_' + some_word
        
    return some_word

def pad_rear(some_word):
    while len(some_word) < max_word_length:
        some_word += '_'
        
    return some_word

words = [only_letters(wrd) for wrd in words if 2 < len(wrd) < 10]
    
print('Found {} words'.format(len(words)))
print(words[7000])

Found 8124 words
sandwich


In [4]:
# create program to generate pig latin
def make_piglatin(some_word):    
    vowels = ['a', 'e', 'i', 'o', 'u']
    ay = 'ay'

    if some_word[0] in vowels:
        new_word = '>' + some_word + ay
    else:
        new_word = '>' + some_word[1:] + some_word[0] + ay
        
    new_word = pad_rear(new_word)
    old_word = pad_front(some_word)
        
    return old_word, new_word

# test it
old_word, new_word = make_piglatin(words[-7000])
print(old_word, new_word)
print(words[-7000], make_piglatin(words[-7000]))

_____cameras >amerascay__
cameras ('_____cameras', '>amerascay__')


In [5]:
# get dictionaries so we can convert from letters to index and index to letters
chars = string.ascii_letters + '>_'

char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for i, ch in enumerate(chars)}

vocab_size = len(chars)

In [6]:
# set up the place holders for our computational graph
inputs = tf.placeholder(shape=[1, None], dtype=tf.int32, name='input')
decoder_inputs = tf.placeholder(shape=[None, None], dtype=tf.int32, name='decoder_input')
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.int32, name='targets')

# create an initializer to init our weight matricies
init = tf.random_normal_initializer(stddev=0.1)

In [7]:
# set up the embedding layer
embeddings = tf.get_variable("embeddings", [vocab_size, embed_size], initializer=init)

with tf.device('/cpu:0'), tf.name_scope("input_embedding"):
    encoder_embedding = tf.nn.embedding_lookup(embeddings, inputs)

with tf.device('/cpu:0'), tf.name_scope("output_embedding"):
    decoder_embedding = tf.nn.embedding_lookup(embeddings, decoder_inputs)

In [8]:
projection_layer = tf.layers.Dense(vocab_size, use_bias=False)


# create the encoder LSTM
with tf.variable_scope('encoder') as enc_scope:
    lstm_encoder = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    _, encoder_last_state = tf.nn.dynamic_rnn(lstm_encoder, inputs=encoder_embedding, 
                                              dtype=tf.float32, time_major=False)
    
with tf.variable_scope('decoder') as dec_scope:
    lstm_decoder = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #decoder_out, _ = tf.nn.dynamic_rnn(lstm_decoder, inputs=decoder_embedding, 
    #                                   initial_state=encoder_last_state, dtype=tf.float32)
    helper = seq2seq.TrainingHelper(inputs=decoder_embedding, sequence_length=[11],
                                    time_major=False)
    decoder = seq2seq.BasicDecoder(lstm_decoder, helper, encoder_last_state, 
                                   output_layer=projection_layer)
    outputs, _, _ = seq2seq.dynamic_decode(decoder, output_time_major=False,
                                           impute_finished=True, maximum_iterations=20)
    
# get the output from the decoder
#logits = tf.layers.Dense(vocab_size, activation_fn=None)
#piglatin_pred = tf.nn.softmax(logits)
logits = outputs.rnn_output
predictions = outputs.sample_id

In [9]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=logits)
loss = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0003).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
logits.get_shape().as_list()

[1, None, 54]

In [11]:
init = tf.global_variables_initializer()

# get test words
words = shuffle(words)
test_words = words[:10]
words = words[10:]

def one_hot(v):
    return np.eye(vocab_size)[v]

with tf.Session() as sess:
    sess.run(init)
    for e in range(epochs):
        words = shuffle(words)

        input_word, targ_word = make_piglatin(words[0])

        input_vals = [char2idx[c] for c in input_word]
        target_vals = [char2idx[c] for c in targ_word[1:]]
        decoder_vals = [char2idx[c] for c in targ_word[:-1]]

        target_vals = one_hot(target_vals)

        input_vals = np.asarray(input_vals).reshape(1, len(input_vals))
        decoder_vals = np.asarray(decoder_vals).reshape(1, len(decoder_vals))

        _, loss_val, train_logits = sess.run([optimizer, loss, logits], feed_dict={inputs: input_vals,
                                                                                   decoder_inputs: decoder_vals,
                                                                                   targets: target_vals})


        if e % 500 == 0:
            print('epoch: {}, loss: {}'.format(e, loss_val))
            
            for i in range(len(test_words)):
                val_word, val_targ = make_piglatin(test_words[i])
                val_input = [char2idx[c] for c in val_word]
                val_dec_in = [char2idx[c] for c in val_targ[:-1]]
                
                val_input = np.asarray(val_input).reshape(1, len(val_input))
                val_dec_in = np.asarray(val_dec_in).reshape(1, len(val_dec_in))
                
                prediction = sess.run(predictions, feed_dict={inputs: val_input,
                                                          decoder_inputs: val_dec_in})

                print("Given: {}, Predicted: {}".format(val_word, ''.join([idx2char[idx] for idx in prediction[0]])))
            print()

epoch: 0, loss: 3.984480142593384
Given: _________doe, Predicted: lllllllllCC
Given: _________yet, Predicted: dddhqNCCCCC
Given: ___respected, Predicted: lldPxrSrlxH
Given: ________trip, Predicted: Bff>cs>CCC_
Given: ____snapshot, Predicted: bbjYUUppUt>
Given: ____minority, Predicted: BCbb>>e>>>>
Given: ____cosmetic, Predicted: bfffjjfWra>
Given: ___forbidden, Predicted: llH>>aalIjj
Given: ______notice, Predicted: llWeWrrjxll
Given: ____ultimate, Predicted: bffeffoWSov

epoch: 500, loss: 1.1736130714416504
Given: _________doe, Predicted: aeaay______
Given: _________yet, Predicted: aaaay______
Given: ___respected, Predicted: eeeettaaay_
Given: ________trip, Predicted: aetaay_____
Given: ____snapshot, Predicted: eeettaaay__
Given: ____minority, Predicted: eeettaaa___
Given: ____cosmetic, Predicted: eeeetaaayy_
Given: ___forbidden, Predicted: eeeetaaay__
Given: ______notice, Predicted: eeeeaaay___
Given: ____ultimate, Predicted: eeeeeaaay__

epoch: 1000, loss: 1.3517146110534668
Given: __

epoch: 9000, loss: 0.441118985414505
Given: _________doe, Predicted: oeday______
Given: _________yet, Predicted: etyay______
Given: ___respected, Predicted: eseecterray
Given: ________trip, Predicted: rittay_____
Given: ____snapshot, Predicted: aasthossay_
Given: ____minority, Predicted: inorttyhay_
Given: ____cosmetic, Predicted: osteciccay_
Given: ___forbidden, Predicted: ommudeedbay
Given: ______notice, Predicted: otioenay___
Given: ____ultimate, Predicted: uttiaateay_

epoch: 9500, loss: 0.39260366559028625
Given: _________doe, Predicted: oeday______
Given: _________yet, Predicted: etyay______
Given: ___respected, Predicted: eseectereay
Given: ________trip, Predicted: rittay_____
Given: ____snapshot, Predicted: aastoetsay_
Given: ____minority, Predicted: inrritycay_
Given: ____cosmetic, Predicted: osteciccay_
Given: ___forbidden, Predicted: ormineencay
Given: ______notice, Predicted: oninenay___
Given: ____ultimate, Predicted: uttiaeeeay_

epoch: 10000, loss: 0.305949866771698
Give

epoch: 18000, loss: 0.07681002467870712
Given: _________doe, Predicted: oeday______
Given: _________yet, Predicted: etyay______
Given: ___respected, Predicted: espectedray
Given: ________trip, Predicted: riptay_____
Given: ____snapshot, Predicted: aasstotsay_
Given: ____minority, Predicted: inoritymay_
Given: ____cosmetic, Predicted: oseeciccay_
Given: ___forbidden, Predicted: ormdddencay
Given: ______notice, Predicted: oticenay___
Given: ____ultimate, Predicted: ultimateay_

epoch: 18500, loss: 0.009101846255362034
Given: _________doe, Predicted: oeday______
Given: _________yet, Predicted: etyay______
Given: ___respected, Predicted: espectessay
Given: ________trip, Predicted: riptay_____
Given: ____snapshot, Predicted: aapstotsay_
Given: ____minority, Predicted: inoritymay_
Given: ____cosmetic, Predicted: ospeticcay_
Given: ___forbidden, Predicted: ormdddelfay
Given: ______notice, Predicted: oticenay___
Given: ____ultimate, Predicted: ultimateay_

epoch: 19000, loss: 0.073141016066074